In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Sac
lat, lon = 38.5816, -121.4944

url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": "2023-01-01",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m",
    "timezone": "America/Los_Angeles"
}

print("Making request to:", url)
print("Parameters:", params)

response = requests.get(url, params=params)
print(f"\nStatus code: {response.status_code}")

data = response.json()

df = pd.DataFrame({
        'time': pd.to_datetime(data['hourly']['time']),
        'temperature_2m': data['hourly']['temperature_2m'],
        'precipitation': data['hourly']['precipitation'],
        'wind_speed_10m': data['hourly']['wind_speed_10m'],
        'relative_humidity_2m': data['hourly']['relative_humidity_2m']
    })
    
df


Making request to: https://archive-api.open-meteo.com/v1/archive
Parameters: {'latitude': 38.5816, 'longitude': -121.4944, 'start_date': '2023-01-01', 'end_date': '2023-12-31', 'hourly': 'temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m', 'timezone': 'America/Los_Angeles'}

Status code: 200


,time,temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m
0,2023-01-01 00:00:00,9.3,0.0,17.7,84
1,2023-01-01 01:00:00,9.2,0.0,20.5,75
2,2023-01-01 02:00:00,9.0,0.0,22.5,68
3,2023-01-01 03:00:00,8.8,0.0,25.0,63
4,2023-01-01 04:00:00,9.0,0.0,28.2,65
...,...,...,...,...,...
8755,2023-12-31 19:00:00,10.3,0.0,8.7,97
8756,2023-12-31 20:00:00,9.9,0.0,7.8,98
8757,2023-12-31 21:00:00,9.6,0.0,9.4,97
8758,2023-12-31 22:00:00,9.4,0.0,11.0,96


In [10]:
import requests
import pandas as pd
from datetime import datetime
import time

class NWSWeatherAPI:
    """
    National Weather Service API Client for California Weather Data
    Documentation: https://www.weather.gov/documentation/services-web-api
    """
    
    def __init__(self, user_agent="CaliforniaWeatherApp (contact@example.com)"):
        self.base_url = "https://api.weather.gov"
        self.headers = {
            "User-Agent": user_agent,
            "Accept": "application/geo+json"
        }
    
    def get_point_metadata(self, latitude, longitude):
        """Get metadata for a point location"""
        url = f"{self.base_url}/points/{latitude},{longitude}"
        
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            properties = data.get('properties', {})
            
            return {
                'gridId': properties.get('gridId'),
                'gridX': properties.get('gridX'),
                'gridY': properties.get('gridY'),
                'forecast_url': properties.get('forecast'),
                'forecast_hourly_url': properties.get('forecastHourly'),
                'observation_stations_url': properties.get('observationStations'),
                'timezone': properties.get('timeZone'),
                'city': properties.get('relativeLocation', {}).get('properties', {}).get('city'),
                'state': properties.get('relativeLocation', {}).get('properties', {}).get('state')
            }
        except Exception as e:
            print(f"Error: {e}")
            return None
    
    def get_forecast(self, latitude, longitude, hourly=False):
        """Get weather forecast"""
        metadata = self.get_point_metadata(latitude, longitude)
        if not metadata:
            return None
        
        forecast_url = metadata['forecast_hourly_url'] if hourly else metadata['forecast_url']
        
        try:
            response = requests.get(forecast_url, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            periods = data.get('properties', {}).get('periods', [])
            df = pd.DataFrame(periods)
            
            if 'startTime' in df.columns:
                df['startTime'] = pd.to_datetime(df['startTime'])
            if 'endTime' in df.columns:
                df['endTime'] = pd.to_datetime(df['endTime'])
            
            # Add location info
            df['city'] = metadata.get('city')
            df['state'] = metadata.get('state')
            df['latitude'] = latitude
            df['longitude'] = longitude
            
            return df
            
        except Exception as e:
            print(f"Error: {e}")
            return None
    
    def get_observations(self, latitude, longitude, limit=50):
        """Get recent weather observations"""
        metadata = self.get_point_metadata(latitude, longitude)
        if not metadata or not metadata.get('observation_stations_url'):
            return None
        
        try:
            # Get observation stations
            stations_response = requests.get(
                metadata['observation_stations_url'], 
                headers=self.headers, 
                timeout=10
            )
            stations_response.raise_for_status()
            stations_data = stations_response.json()
            
            stations = stations_data.get('features', [])
            if not stations:
                print("No observation stations found")
                return None
            
            # Get observations from first station
            station_id = stations[0].get('properties', {}).get('stationIdentifier')
            obs_url = f"{self.base_url}/stations/{station_id}/observations"
            
            obs_response = requests.get(
                obs_url, 
                headers=self.headers, 
                params={'limit': limit},
                timeout=10
            )
            obs_response.raise_for_status()
            obs_data = obs_response.json()
            
            observations = []
            for feature in obs_data.get('features', []):
                props = feature.get('properties', {})
                observations.append({
                    'timestamp': props.get('timestamp'),
                    'station': station_id,
                    'temperature_c': props.get('temperature', {}).get('value'),
                    'dewpoint_c': props.get('dewpoint', {}).get('value'),
                    'wind_direction': props.get('windDirection', {}).get('value'),
                    'wind_speed_kmh': props.get('windSpeed', {}).get('value'),
                    'wind_gust_kmh': props.get('windGust', {}).get('value'),
                    'barometric_pressure_pa': props.get('barometricPressure', {}).get('value'),
                    'visibility_m': props.get('visibility', {}).get('value'),
                    'relative_humidity': props.get('relativeHumidity', {}).get('value'),
                    'precipitation_last_hour_mm': props.get('precipitationLastHour', {}).get('value'),
                    'description': props.get('textDescription')
                })
            
            df = pd.DataFrame(observations)
            if 'timestamp' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Convert to Fahrenheit
            df['temperature_f'] = df['temperature_c'] * 9/5 + 32
            df['dewpoint_f'] = df['dewpoint_c'] * 9/5 + 32
            
            return df
            
        except Exception as e:
            print(f"Error: {e}")
            return None
    
    def get_alerts(self, state="CA"):
        """Get active weather alerts for California"""
        url = f"{self.base_url}/alerts/active"
        params = {'area': state}
        
        try:
            response = requests.get(url, headers=self.headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            alerts = []
            for feature in data.get('features', []):
                props = feature.get('properties', {})
                alerts.append({
                    'event': props.get('event'),
                    'severity': props.get('severity'),
                    'certainty': props.get('certainty'),
                    'urgency': props.get('urgency'),
                    'headline': props.get('headline'),
                    'description': props.get('description'),
                    'instruction': props.get('instruction'),
                    'onset': props.get('onset'),
                    'expires': props.get('expires'),
                    'affected_zones': ', '.join(props.get('affectedZones', []))
                })
            
            df = pd.DataFrame(alerts)
            if not df.empty and 'onset' in df.columns:
                df['onset'] = pd.to_datetime(df['onset'])
                df['expires'] = pd.to_datetime(df['expires'])
            
            return df
            
        except Exception as e:
            print(f"Error: {e}")
            return None


# California Cities Coordinates
california_cities = {
    'Sacramento': (38.5816, -121.4944),
    'Los Angeles': (34.0522, -118.2437),
    'San Francisco': (37.7749, -122.4194),
    'San Diego': (32.7157, -117.1611),
    'Fresno': (36.7378, -119.7871),
    'San Jose': (37.3382, -121.8863),
    'Oakland': (37.8044, -122.2711),
    'Bakersfield': (35.3733, -119.0187)
}

# Initialize API
nws = NWSWeatherAPI()

print("=" * 80)
print("CALIFORNIA WEATHER DATA FROM NWS API")
print("=" * 80)

# Example 1: Get forecast for Sacramento
print("\n1. SACRAMENTO 7-DAY FORECAST")
print("-" * 80)
lat, lon = california_cities['Sacramento']
forecast_df = nws.get_forecast(lat, lon, hourly=False)

if forecast_df is not None:
    print(forecast_df[['name', 'startTime', 'temperature', 'windSpeed', 'shortForecast']].head(10))
    print(f"Done")

# Example 2: Get hourly forecast for Los Angeles
print("\n2. LOS ANGELES HOURLY FORECAST")
print("-" * 80)
lat, lon = california_cities['Los Angeles']
hourly_df = nws.get_forecast(lat, lon, hourly=True)

if hourly_df is not None:
    print(hourly_df[['startTime', 'temperature', 'windSpeed', 'shortForecast']].head(12))
    print(f"Done")

# Example 3: Get recent observations for San Francisco
print("\n3. SAN FRANCISCO RECENT OBSERVATIONS")
print("-" * 80)
lat, lon = california_cities['San Francisco']
obs_df = nws.get_observations(lat, lon, limit=24)

if obs_df is not None:
    print(obs_df[['timestamp', 'temperature_f', 'wind_speed_kmh', 'relative_humidity', 'description']].head(10))
    print(f"Done")

# Example 4: Get active weather alerts for California
print("\n4. CALIFORNIA ACTIVE WEATHER ALERTS")
print("-" * 80)
alerts_df = nws.get_alerts(state="CA")

if alerts_df is not None and not alerts_df.empty:
    print(f"\n⚠️  {len(alerts_df)} active alerts")
    print(alerts_df[['event', 'severity', 'headline']].head())
    print(f"\n Done")
else:
    print("No active alerts for California")

# Example 5: Get forecast for multiple cities
print("\n5. MULTI-CITY FORECAST COMPARISON")
print("-" * 80)

all_forecasts = []
for city, (lat, lon) in list(california_cities.items())[:5]:
    print(f"Fetching {city}...", end=" ")
    df = nws.get_forecast(lat, lon, hourly=False)
    if df is not None:
        df['city'] = city
        all_forecasts.append(df)
        print("✓")
    else:
        print("✗")
    time.sleep(0.5)  # Be polite to the API

print("\n" + "=" * 80)
print("Done")
print("=" * 80)

CALIFORNIA WEATHER DATA FROM NWS API

1. SACRAMENTO 7-DAY FORECAST
--------------------------------------------------------------------------------
              name                 startTime  temperature   windSpeed  \
0   This Afternoon 2025-11-22 13:00:00-08:00           60       3 mph   
1          Tonight 2025-11-22 18:00:00-08:00           44       2 mph   
2           Sunday 2025-11-23 06:00:00-08:00           59       2 mph   
3     Sunday Night 2025-11-23 18:00:00-08:00           44       2 mph   
4           Monday 2025-11-24 06:00:00-08:00           61  2 to 6 mph   
5     Monday Night 2025-11-24 18:00:00-08:00           41       5 mph   
6          Tuesday 2025-11-25 06:00:00-08:00           58       3 mph   
7    Tuesday Night 2025-11-25 18:00:00-08:00           42       2 mph   
8        Wednesday 2025-11-26 06:00:00-08:00           59       2 mph   
9  Wednesday Night 2025-11-26 18:00:00-08:00           42       2 mph   

                      shortForecast  
0         

In [12]:
forecast_df

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,windSpeed,windDirection,icon,shortForecast,detailedForecast,city,state,latitude,longitude
0,1,This Afternoon,2025-11-22 13:00:00-08:00,2025-11-22 18:00:00-08:00,True,60,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}",3 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny. High near 60, with temperatures falling...",Sacramento,CA,38.5816,-121.4944
1,2,Tonight,2025-11-22 18:00:00-08:00,2025-11-23 06:00:00-08:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 1}",2 mph,SE,https://api.weather.gov/icons/land/night/fog?s...,Widespread Fog,"Widespread fog after 10pm. Partly cloudy, with...",Sacramento,CA,38.5816,-121.4944
2,3,Sunday,2025-11-23 06:00:00-08:00,2025-11-23 18:00:00-08:00,True,59,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 1}",2 mph,WNW,https://api.weather.gov/icons/land/day/fog/bkn...,Widespread Fog then Partly Sunny,Widespread fog before 11am. Partly sunny. High...,Sacramento,CA,38.5816,-121.4944
3,4,Sunday Night,2025-11-23 18:00:00-08:00,2025-11-24 06:00:00-08:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}",2 mph,WNW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 44. West nort...",Sacramento,CA,38.5816,-121.4944
4,5,Monday,2025-11-24 06:00:00-08:00,2025-11-24 18:00:00-08:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}",2 to 6 mph,NNW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 61. North north...",Sacramento,CA,38.5816,-121.4944
5,6,Monday Night,2025-11-24 18:00:00-08:00,2025-11-25 06:00:00-08:00,False,41,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}",5 mph,NNW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 41.",Sacramento,CA,38.5816,-121.4944
6,7,Tuesday,2025-11-25 06:00:00-08:00,2025-11-25 18:00:00-08:00,True,58,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}",3 mph,N,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 58.",Sacramento,CA,38.5816,-121.4944
7,8,Tuesday Night,2025-11-25 18:00:00-08:00,2025-11-26 06:00:00-08:00,False,42,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 1}",2 mph,ESE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 42.",Sacramento,CA,38.5816,-121.4944
8,9,Wednesday,2025-11-26 06:00:00-08:00,2025-11-26 18:00:00-08:00,True,59,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 1}",2 mph,NNW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 59.",Sacramento,CA,38.5816,-121.4944
9,10,Wednesday Night,2025-11-26 18:00:00-08:00,2025-11-27 06:00:00-08:00,False,42,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 1}",2 mph,N,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 42.",Sacramento,CA,38.5816,-121.4944


In [13]:
obs_df

,timestamp,station,temperature_c,dewpoint_c,wind_direction,wind_speed_kmh,wind_gust_kmh,barometric_pressure_pa,visibility_m,relative_humidity,precipitation_last_hour_mm,description,temperature_f,dewpoint_f
0,2025-11-23 01:43:00+00:00,SFOC1,11.78,10.20,None,None,None,None,None,90.043138,None,,53.204,50.360
1,2025-11-23 00:43:00+00:00,SFOC1,12.17,9.72,None,None,None,None,None,84.985267,None,,53.906,49.496
2,2025-11-22 23:43:00+00:00,SFOC1,13.49,9.92,None,None,None,None,None,79.004958,None,,56.282,49.856
3,2025-11-22 22:43:00+00:00,SFOC1,15.63,7.39,None,None,None,None,None,57.991078,None,,60.134,45.302
4,2025-11-22 21:43:00+00:00,SFOC1,17.39,9.79,None,None,None,None,None,60.992121,None,,63.302,49.622
5,2025-11-22 20:43:00+00:00,SFOC1,18.07,7.48,None,None,None,None,None,49.983123,None,,64.526,45.464
6,2025-11-22 19:43:00+00:00,SFOC1,17.43,7.18,None,None,None,None,None,50.983104,None,,63.374,44.924
7,2025-11-22 18:43:00+00:00,SFOC1,15.94,9.13,None,None,None,None,None,63.975002,None,,60.692,48.434
8,2025-11-22 17:43:00+00:00,SFOC1,13.66,10.08,None,None,None,None,None,78.976970,None,,56.588,50.144
9,2025-11-22 16:43:00+00:00,SFOC1,12.83,10.89,None,None,None,None,None,87.994015,None,,55.094,51.602
